In [4]:
from experta import *

class WhiteMove(Fact):
    number = Field(int, mandatory=True)  # mandatory means required
    white = Field(str, mandatory=True)

class BlackMove(Fact):
    number = Field(int, mandatory=True)
    black = Field(str, mandatory=True)

class Branch(Fact):
    name = Field(str, mandatory=True)
    moves = Field(list, default=[])

class Opening(Fact):
    name = Field(str, mandatory=True)
    branches = Field(list, default=[])
    color = Field(str, default='both')

class Path(Fact):
    """My playing path"""
    moves = Field(list, default=[])

class Questionnaire(Fact):
    """Questions to determine playing style."""
    question = Field(str, mandatory=True)

class CertaintyFactor(Fact):
    """Chess related facts with certainty factors."""
    name = Field(str, mandatory=True)
    cf = Field(float, default=100.0)

class ChessOpeningEngine(KnowledgeEngine):

    @DefFacts()
    def _initial_action(self):
        yield Fact(end=False)
        yield Path(moves=[])
        yield Fact(possible_moves=[])
        # Openings
        yield Opening(name='Ruy Lopez', color='white', branches=[
            Branch(name='Main Line', moves=[
                WhiteMove(number=0, name='e4'),
                BlackMove(number=0, name='e5'),
                WhiteMove(number=1, name='Nf3'),
                BlackMove(number=1, name='Nc6'),
                WhiteMove(number=2, name='Bb5'),
                BlackMove(number=2, name='a6'), # old steinitz, bird
                WhiteMove(number=3, name='Ba4'),
                BlackMove(number=3, name='d6'),
                WhiteMove(number=4, name='O-O'),
                BlackMove(number=4, name='b5'),
                WhiteMove(number=5, name='Bb3'),
                BlackMove(number=5, name='Bg4'),
                WhiteMove(number=6, name='h3'),
                BlackMove(number=6, name='Bf3'),
                WhiteMove(number=7, name='Qf3')
            ]),
            Branch(name='The Old Steinitz Defense', moves=[
                BlackMove(number=2, name='d6'),
                WhiteMove(number=3, name='d4'),
                BlackMove(number=3, name='exd4'),
                WhiteMove(number=4, name='Nd4'),
                BlackMove(number=4, name='Bb7'),
                WhiteMove(number=5, name='Nc3'),
                BlackMove(number=5, name='Nd4'),
                WhiteMove(number=6, name='Qd4'),
                BlackMove(number=6, name='Bb5'),
                WhiteMove(number=7, name='Nxb5'),
                BlackMove(number=7, name='Nf6'),
                WhiteMove(number=8, name='e5'),
                BlackMove(number=8, name='exe5'),
                WhiteMove(number=9, name='Qe5'),
            ]),
            Branch(name='The Bird\'s Defense', moves=[
                BlackMove(number=2, name='Nd4'),
                WhiteMove(number=3, name='NxNd4'),
            ]),
        ])
        yield Opening(name='Caro-Kann Defense', color='black', branches=[
            Branch(name='Main Line', moves=[
                WhiteMove(number=1, name='e4'),
                BlackMove(number=1, name='c6'),
                WhiteMove(number=2, name='d4'),
                BlackMove(number=2, name='d5')
            ]),
        ])
        yield Opening(name='Queen\'s Gambit', color='white', branches=[])

    # Start
    @Rule(salience=100)
    def start(self):
        color = input("Do you want to play as White or Black? ").strip().lower()
        self.declare(Fact(color=color))

    # List Openings
    @Rule(Fact(color=MATCH.mycolor))
    def list_openings(self, mycolor):
        print(f"You chose to play as {mycolor}.\n")
        print(f"{mycolor} Openings:")

        # Collect and print valid openings
        valid_openings = self.collect_and_print_openings(mycolor)
        # Get the chosen opening
        chosen_opening = self.get_valid_opening(valid_openings)
        print()
        self.declare(Fact(chosen_opening=chosen_opening))

    # declare first move
    @Rule(Fact(chosen_opening=MATCH.name),
          AS.opening << Opening(name=MATCH.name),
          salience=10
        )
    def basic_declare(self, opening):
        self.declare(Fact(current_branch=0))
        self.declare(Fact(current_move=0))

        # For testing
        # self.print_branch_moves(opening)

    # Ruy Lopez - The Old Steinitz Defense
    @Rule(AS.opening << Opening(name='Ruy Lopez'),
          AS.branch_fact << Fact(current_branch=0),
          AS.move_fact << Fact(current_move=5),
          AS.possible_fact << Fact(possible_moves=MATCH.possible_moves),
          TEST(lambda possible_moves: len(possible_moves) == 0),
          salience=5
        )
    def old_steintiz(self, opening, branch_fact, move_fact, possible_fact, possible_moves):
        next_move = opening["branches"][0]["moves"][5]["name"]
        other_move = opening["branches"][1]["moves"][0]["name"]
        new_possible_moves = list(possible_moves)
        new_possible_moves.append({"name":next_move, "branch_index": 0, "index": 5})
        new_possible_moves.append({"name":other_move, "branch_index": 1, "index": 0})
        self.modify(possible_fact, possible_moves=new_possible_moves)
        print("Ruy Lopez - The Old Steinitz Defense")

    # Ruy Lopez - The Bird's Defense
    @Rule(AS.opening << Opening(name='Ruy Lopez'),
          AS.branch_fact << Fact(current_branch=0),
          AS.move_fact << Fact(current_move=5),
          AS.possible_fact << Fact(possible_moves=MATCH.possible_moves),
          TEST(lambda possible_moves: len(possible_moves) == 2),
          salience=5
        )
    def bird(self, opening, branch_fact, move_fact, possible_fact, possible_moves):
        other_move = opening["branches"][2]["moves"][0]["name"]
        new_possible_moves = list(possible_moves)
        new_possible_moves.append({"name":other_move, "branch_index": 2, "index": 0})
        self.modify(possible_fact, possible_moves=new_possible_moves)
        print("Ruy Lopez - The Bird's Defense")

    # Auto-Play when only one move is available
    @Rule(Fact(chosen_opening=MATCH.name),
          AS.opening << Opening(name=MATCH.name),
          Fact(current_branch=MATCH.branch_index),
          AS.move_fact << Fact(current_move=MATCH.move_index),
          AS.endFact << Fact(end=L(False)),
          AS.possible_fact << Fact(possible_moves=MATCH.possible_moves),
          TEST(lambda possible_moves: len(possible_moves) <= 1),
          salience=1
    )
    def auto_play(self, opening, branch_index, move_index, move_fact, endFact, possible_fact):
        moves = opening["branches"][branch_index]["moves"]

        if move_index < len(moves):
            move = moves[move_index]
            if move_index % 2 == 0:
                print(f"White moves: {move['name']}", flush=True)
            else:
                print(f"Black moves: {move['name']}", flush=True)

            self.modify(move_fact, current_move=move_index + 1)
            self.modify(possible_fact, possible_moves=[])
    
        else:
            print("No more moves in the branch.")
            self.modify(endFact, end=True)

    # User-Play when multiple moves are available
    @Rule(Fact(chosen_opening=MATCH.name),
          AS.opening << Opening(name=MATCH.name),
          AS.branch_fact << Fact(current_branch=MATCH.branch_index),
          AS.move_fact << Fact(current_move=MATCH.move_index),
          AS.possible_fact << Fact(possible_moves=MATCH.possible_moves),
          TEST(lambda possible_moves: len(possible_moves) > 1),
          salience=1
    )
    def choose_move(self, move_fact, branch_fact, possible_fact, possible_moves):
        print("Multiple possible moves available:")
        names = []
        for move in possible_moves:
            print(f"{move['name']}")
            names.append(move['name'])

        while True:
            choice = input("Choose your move: ")
            if choice in names:
                chosen_move = next(move for move in possible_moves if move['name'] == choice)
                print(f"Your choice: {chosen_move['name']}", flush=True)
                self.modify(possible_fact, possible_moves=[])
                self.modify(move_fact, current_move=chosen_move['index'] + 1) # get the chosen move index
                self.modify(branch_fact, current_branch=chosen_move['branch_index']) # get the chosen branch index
                break
            else:
                print("Invalid choice. Please select again.")
     

    # Helper functions
    def print_branch_moves(self, opening):
        branches = opening['branches']
        for branch in branches:
            print(f'\n{opening["name"]} - {branch["name"]}:')
            for move in branch["moves"]:
                print(f"{move['number']}.{move['name']}")               
        print()

    def collect_and_print_openings(self, mycolor):
        valid_openings = []
        for opening in self.facts.values():
            if isinstance(opening, Opening) and opening["color"] in (mycolor, "both"):
                print(opening["name"], flush=True)
                valid_openings.append(opening["name"])
        return valid_openings

    def get_valid_opening(self, valid_openings):
        while True:
            chosen_opening = input('Choose an opening: ')
            if chosen_opening in valid_openings:
                return chosen_opening
            else:
                print("Invalid opening. Please choose from the listed openings.")

engine = ChessOpeningEngine()
engine.reset()
engine.run()
engine.facts

You chose to play as white.

white Openings:
Ruy Lopez
Queen's Gambit

White moves: e4
Black moves: e5
White moves: Nf3
Black moves: Nc6
White moves: Bb5
Ruy Lopez - The Old Steinitz Defense
Ruy Lopez - The Bird's Defense
Multiple possible moves available:
a6
d6
Nd4
Invalid choice. Please select again.
Your choice: Nd4
Black moves: NxNd4
No more moves in the branch.


FactList([(0, InitialFact()),
          (2, Path(moves=frozenlist([]))),
          (4,
           Opening(name='Ruy Lopez', color='white', branches=frozenlist([<frozendict {'name': 'Main Line', 'moves': frozenlist([<frozendict {'number': 0, 'name': 'e4'}>, <frozendict {'number': 0, 'name': 'e5'}>, <frozendict {'number': 1, 'name': 'Nf3'}>, <frozendict {'number': 1, 'name': 'Nc6'}>, <frozendict {'number': 2, 'name': 'Bb5'}>, <frozendict {'number': 2, 'name': 'a6'}>, <frozendict {'number': 3, 'name': 'Ba4'}>, <frozendict {'number': 3, 'name': 'd6'}>, <frozendict {'number': 4, 'name': 'O-O'}>, <frozendict {'number': 4, 'name': 'b5'}>, <frozendict {'number': 5, 'name': 'Bb3'}>, <frozendict {'number': 5, 'name': 'Bg4'}>, <frozendict {'number': 6, 'name': 'h3'}>, <frozendict {'number': 6, 'name': 'Bf3'}>, <frozendict {'number': 7, 'name': 'Qf3'}>])}>, <frozendict {'name': 'The Old Steinitz Defense', 'moves': frozenlist([<frozendict {'number': 2, 'name': 'd6'}>, <frozendict {'number': 3, 'nam

**Chess Openings**
